## Vanishing/Exploding Gradients

In [1]:
from tensorflow import keras

In [2]:
keras.layers.Dense(10, activation="relu", 
                    kernel_initializer="he_normal")

In [4]:
he_avg_init = keras.initializers.VarianceScaling(scale=2, 
                                                mode="fan_avg", 
                                                distribution="uniform")
keras.layers.Dense(10, activation="relu", 
                    kernel_initializer=he_avg_init)

In [5]:
# using leaky relu
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
keras.layers.Dense(10, activation=leaky_relu, 
                    kernel_initializer="he_normal")

In [6]:
keras.layers.Dense(10, activation="selu", 
                    kernel_initializer="lecun_normal")

In [8]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300, activation="elu", 
                        kernel_initializer="he_normal"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(100, activation="elu", 
                        kernel_initializer="he_normal"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(10, activation="softmax")
])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_4 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_5 (Batc  (None, 100)             

In [10]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization_3/gamma:0', True),
 ('batch_normalization_3/beta:0', True),
 ('batch_normalization_3/moving_mean:0', False),
 ('batch_normalization_3/moving_variance:0', False)]

In [11]:
model.layers[1].updates

C:\Users\SG704\AppData\Local\Temp/ipykernel_12816/3873162892.py:1: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  model.layers[1].updates


[]

In [2]:
from tensorflow import keras

In [3]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300, 
        kernel_initializer="he_normal", 
        use_bias=False), 
    keras.layers.BatchNormalization(), 
    keras.layers.Activation("elu"), 
    keras.layers.Dense(100, 
        kernel_initializer="he_normal", 
        use_bias=False), 
    keras.layers.BatchNormalization(), 
    keras.layers.Activation("elu"), 
    keras.layers.Dense(10, activation="softmax")
])

In [4]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

## Reusing Pretrained Layers

In [54]:
from tensorflow import keras
import numpy as np

def get_data_without_5_6(data, labels):
    new_data, new_labels = [], []
    for i, label in enumerate(labels):
        if (label != 5) & (label != 6):
            if label > 5:
                new_labels.append(labels[i]-2)
            else:
                new_labels.append(labels[i])
            new_data.append(data[i])
    return np.array(new_data), np.array(new_labels)
        
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_A, y_train_A = get_data_without_5_6(X_train, y_train)
X_test_A, y_test_A = get_data_without_5_6(X_test, y_test)
X_train_A, X_val_A, y_train_A, y_val_A = X_train_A[:30000], X_train_A[30000:], y_train_A[:30000], y_train_A[30000:]

In [55]:


model_A = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(100, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(8, activation="softmax")
])
model_A.compile(loss="sparse_categorical_crossentropy", 
                optimizer="sgd", 
                metrics="accuracy")
model_A.fit(X_train_A, y_train_A, epochs=100, 
            validation_data=(X_val_A, y_val_A), 
            callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                     patience=10, 
                                                     restore_best_weights=True)])

Epoch 1/100
938/938 [==============================] - 5s 4ms/step - loss: 0.3892 - accuracy: 0.8676 - val_loss: 0.2706 - val_accuracy: 0.9073
Epoch 2/100
938/938 [==============================] - 4s 4ms/step - loss: 0.2725 - accuracy: 0.9046 - val_loss: 0.2450 - val_accuracy: 0.9166
Epoch 3/100
938/938 [==============================] - 4s 4ms/step - loss: 0.2371 - accuracy: 0.9168 - val_loss: 0.2353 - val_accuracy: 0.9197
Epoch 4/100
938/938 [==============================] - 3s 4ms/step - loss: 0.2207 - accuracy: 0.9227 - val_loss: 0.2266 - val_accuracy: 0.9239
Epoch 5/100
938/938 [==============================] - 3s 3ms/step - loss: 0.2028 - accuracy: 0.9296 - val_loss: 0.2209 - val_accuracy: 0.9247
Epoch 6/100
938/938 [==============================] - 5s 5ms/step - loss: 0.1880 - accuracy: 0.9342 - val_loss: 0.2229 - val_accuracy: 0.9253
Epoch 7/100
938/938 [==============================] - 3s 4ms/step - loss: 0.1788 - accuracy: 0.9373 - val_loss: 0.2173 - val_accuracy: 0.9248

In [30]:
model_A.evaluate(X_test_A, y_test_A)

250/250 [==============================] - 1s 1ms/step - loss: 0.2616 - accuracy: 0.9186


[0.2615906894207001, 0.918624997138977]

In [44]:
def get_data_withonly_5_6(data, labels):
    new_data, new_labels = [], []
    for i, label in enumerate(labels):
        if label == 5:
            new_data.append(data[i])
            new_labels.append(1)
        elif label == 6:
            new_data.append(data[i])
            new_labels.append(0)
    return np.array(new_data), np.array(new_labels)

X_train_B, y_train_B = get_data_withonly_5_6(X_train, y_train)
X_test_B, y_test_B = get_data_withonly_5_6(X_test, y_test)
X_train_B, X_val_B, y_train_B, y_val_B = X_train_B[:10000], X_train_B[10000:], y_train_B[:10000], y_train_B[10000:]

In [45]:
model_B = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(100, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(10, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(1, activation="sigmoid")
])
model_B.compile(loss="binary_crossentropy", 
                optimizer="sgd", 
                metrics="accuracy")
model_B.fit(X_train_B, y_train_B, epochs=100, 
            validation_data=(X_val_B, y_val_B), 
            callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                     patience=10, 
                                                     restore_best_weights=True)])

Epoch 1/100
313/313 [==============================] - 2s 3ms/step - loss: 0.1158 - accuracy: 0.9719 - val_loss: 0.0414 - val_accuracy: 0.9960
Epoch 2/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0409 - accuracy: 0.9937 - val_loss: 0.0237 - val_accuracy: 0.9955
Epoch 3/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0272 - accuracy: 0.9964 - val_loss: 0.0173 - val_accuracy: 0.9965
Epoch 4/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0212 - accuracy: 0.9956 - val_loss: 0.0146 - val_accuracy: 0.9970
Epoch 5/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0179 - accuracy: 0.9970 - val_loss: 0.0136 - val_accuracy: 0.9960
Epoch 6/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0130 - accuracy: 0.9980 - val_loss: 0.0124 - val_accuracy: 0.9960
Epoch 7/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0107 - accuracy: 0.9983 - val_loss: 0.0118 - val_accuracy: 0.9965

In [46]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.0059 - accuracy: 0.9980


[0.005876780953258276, 0.9980000257492065]

In [56]:
model_A.evaluate(X_test_A, y_test_A)

250/250 [==============================] - 1s 2ms/step - loss: 0.2518 - accuracy: 0.9258


[0.25178515911102295, 0.9257500171661377]

In [58]:
model_B_on_A = keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [59]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [60]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", 
                     optimizer="sgd", 
                     metrics=["accuracy"])


In [61]:
history_b_on_A = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, 
                                  validation_data=(X_val_B, y_val_B))

Epoch 1/4
313/313 [==============================] - 2s 4ms/step - loss: 0.1218 - accuracy: 0.9689 - val_loss: 0.0540 - val_accuracy: 0.9915
Epoch 2/4
313/313 [==============================] - 1s 2ms/step - loss: 0.0376 - accuracy: 0.9948 - val_loss: 0.0357 - val_accuracy: 0.9945
Epoch 3/4
313/313 [==============================] - 1s 3ms/step - loss: 0.0270 - accuracy: 0.9957 - val_loss: 0.0292 - val_accuracy: 0.9960
Epoch 4/4
313/313 [==============================] - 1s 2ms/step - loss: 0.0223 - accuracy: 0.9963 - val_loss: 0.0258 - val_accuracy: 0.9960


In [ ]:
# for layer in model_B_on_A.layers